In [1]:
 '''
Author information:
Joey R. Muffoletto
University of Texas at Austin
Autonomous Systems Group
jrmuff@utexas.edu
'''

In [2]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt


In [3]:
NUM_VERTIHUBS = 10 #dependent on trip_data
PORTS_PER_VERTIHUB = 3
FLIGHT_SPEED = 60 #m/s

In [4]:
'''
Trip data formatting
'''
trip_data = pd.read_csv('data/Realtime/OpsLimits/trips.csv')
trip_data.drop(trip_data[trip_data.OriginVertihub == trip_data.DestinationVertihub].index, inplace=True) #drop trips where the origin and destination are the same vertiport, reduced from 4801 trips to 1692 with 10 vertiports
trip_data

,Time,OriginVertihub,OriginVertiport,DestinationVertihub,DestinationVertiport
5,0,3,13,1,17
8,0,6,11,1,17
9,0,2,0,6,26
11,0,1,1,7,12
12,0,5,5,1,8
...,...,...,...,...,...
4789,0,3,9,6,21
4792,0,6,11,1,17
4793,0,5,29,2,25
4795,0,6,2,3,13


In [5]:
vertiports_data = pd.read_csv('data/Realtime/OpsLimits/vertiports.csv')

In [6]:
from math import cos, asin, sqrt, pi

# def dist(x1, y1, x2, y2):
#     return math.sqrt((y2-y1)**(y2-y1) + (x2-x1)**(x2-x1))
def ll_distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return 12742 * asin(sqrt(a)) #2*R*asin...   
    

In [7]:
'''
input statistics
'''

arrivals_per_vertihub = [[] for i in range(NUM_VERTIHUBS)]
flight_popularity = dict()
# get the trip arrival time per vertihub
for index, trip in trip_data.iterrows():
    #calculate time of arrival (begin at trip.time), calculated between vertiports
    origin_port = vertiports_data.iloc[trip['OriginVertiport']]
    destination_port = vertiports_data.iloc[trip['DestinationVertiport']]
    if((origin_port['vertiport'],destination_port['vertiport']) not in flight_popularity.keys()):
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] = 1
    else:
        flight_popularity[(origin_port['vertiport'], destination_port['vertiport'])] += 1
    #trip distance in kilometers
    dist = ll_distance(origin_port['lat'], origin_port['long'], destination_port['lat'], destination_port['long'])
    travel_time = (dist*1000)/FLIGHT_SPEED # (km * 1000)/(m/s) -> seconds
    arrival_time = trip['Time'] + travel_time #takeoff time + travel_time = arrival time
    #add arrival time to vertihub
    arrivals_per_vertihub[trip['DestinationVertihub']].append(arrival_time)

# for each list of vertihub, calculate the minimum, average, and maximum distance between arrivals
differences_per_vertihub = []
# simulated_arrivals
for arrivals_list in arrivals_per_vertihub:
    sorted_arrivals = sorted(arrivals_list)
    differences = []
    for index in range(len(sorted_arrivals)-1):
        differences.append(sorted_arrivals[index+1] - sorted_arrivals[index])
    print(sorted_arrivals)
    differences_per_vertihub.append(sorted(differences))

for index, differences in enumerate(differences_per_vertihub):
    print('vertihub ' + str(index) + ' has minimum arrival difference of ' + str(differences[0]))
    print('vertihub ' + str(index) + ' has average arrival difference of ' + str(sum(differences)/len(differences)))
    print('vertihub ' + str(index) + ' has maximum arrival difference of ' + str(differences[len(differences)-1]))
    print('vertihub ' + str(index) + ' has ' + str(len(differences)) + ' trips')
    print('')
   

formatted_flight_popularity = [(flight_popularity[key], key) for key in flight_popularity.keys()]
print((sorted(formatted_flight_popularity)))
print(len(formatted_flight_popularity))
avg = sum([num for num, thing in formatted_flight_popularity])/len(formatted_flight_popularity)
print(avg)
    
    

[588.2580132840834, 633.8552982119246, 633.8552982119246, 782.1476763768136, 782.1476763768136, 782.1476763768136, 782.1476763768136, 782.1476763768136, 782.1476763768136, 915.6006991189424, 915.6006991189424, 1015.2430576196089, 1015.2430576196089, 1015.2430576196089, 1015.2430576196089, 1015.2430576196089, 1015.2430576196089, 1064.278583479401, 1173.4080541395551, 1173.4080541395551, 1293.674649089234, 1307.1027007449798, 1460.14149531695, 1476.1974206353211, 1476.1974206353211, 1476.1974206353211, 1476.1974206353211, 1832.3671202564583, 2073.4014891671677, 2073.4014891671677]
[286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.26514556522454, 286.265145565